# Local Area Unemployment Statistics  
This notebook pulls data for all counties of concern, and places with population over 25,000 that have unemployment statistics recorded by the Bureau of Labor Statistics. This notebook writes out two files - one with the most recent annual average unemployment rate, and one for month over month unadjusted unemployment from 2019 to the most recent estimate.

## Series Codes:
https://www.bls.gov/help/hlpforma.htm#AP

In [32]:
import pandas as pd
import json
import requests

In [33]:
def get_laus(series, start, end):
    headers = {'Content-Type': 'application/json'}
    data = json.dumps({"seriesid": series,"startyear":"%d" % (start), "endyear":"%d" % (end), "annualaverage": True})
    p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
    json_data = json.loads(p.text)
    try:
        df = pd.DataFrame()
        for series in json_data['Results']['series']:
            df_initial = pd.DataFrame(series)
            series_col = df_initial['seriesID'][0]
            for i in range(0, len(df_initial) - 1):
                df_row = pd.DataFrame(df_initial['data'][i])
                df_row['seriesID'] = series_col
                if 'code' not in str(df_row['footnotes']): 
                    df_row['footnotes'] = ''
                else:
                    df_row['footnotes'] = str(df_row['footnotes']).split("'code': '",1)[1][:1]
                df = df.append(df_row, ignore_index=True)
        return df
    except:
        json_data['status'] == 'REQUEST_NOT_PROCESSED'
        print('BLS API has given the following Response:', json_data['status'])
        print('Reason:', json_data['message'])

In [34]:
#contains all places with threshold population as of October, 2022: https://download.bls.gov/pub/time.series/la/la.area
#split counties and places because it's easier to adjust the geoids to match census ones that way, you could run them all together if you wanted
counties = ['CN4702100000000', #Cheatham, 
            'CN4703700000000', #Davidson
            'CN4704300000000', #Dickson, 
            'CN4708300000000', #Houston
            'CN4708500000000', #Humphreys
            'CN4711100000000', #Macon
            'CN4711900000000', #Maury
            'CN4712500000000', #Montgomery
            'CN4714700000000', #Robertson
            'CN4714900000000', #Rutherford
            'CN4716100000000', #Stewart
            'CN4716500000000', #Sumner
            'CN4716900000000', #Trousdale
            'CN4718700000000', #Williamson
            'CN4718900000000', 
            'CN2100300000000', #Allen (KY)
            'CN2121300000000'] #Simpson (KY)]  #Wilson
places = ['CT4708280000000', #Brentwood city, Tennessee: Williamson
          'CT4715160000000', #Clarksville city, Tennessee: Montgomery
          'CT4716540000000', #Columbia city, Tennessee: Maury
          'CT4727740000000', #Franklin city, Tennessee: Williamson
          'CT4728540000000', #Gallatin city, Tennessee: Sumner
          'CT4733280000000', #Hendersonville city, Tennessee: Sumner
          'CT4741200000000', #La Vergne city, Tennessee: Rutherford
          'CT4741520000000', #Lebanon city, Tennessee: Wilson
          'CT4750780000000', #Mount Juliet city, Tennessee: Wilson
          'CT4751560000000', #Murfreesboro city, Tennessee: Rutherford
          'CT4752004000000', #Nashville-Davidson metropolitan government (balance): Davidson
          'CT4769420000000', #Smyrna town, Tennessee: Rutherford
          'CT4770580000000'] #Spring Hill city, Tennessee: Maury/Williamson

In [35]:
geos = pd.read_csv('../../Data Downloads/geofips.csv')

In [36]:
FIPSCounties = []
for i in counties:
    FIPSCounties.append('LAU{}03'.format(i))

In [37]:
FIPSPlaces = []
for i in places:
    FIPSPlaces.append('LAU{}03'.format(i))

In [38]:
dfco = get_laus(FIPSCounties, 2019, 2022)

In [39]:
test = dfco['seriesID'].str.split(pat = 'CN', expand = True)
test = test[1]
test = test.str[:5]
dfco['GEO_ID'] = '0500000US' + test

In [9]:
dfpl = get_laus(sFIPSPlaces, 2019, 2022)

In [10]:
test = dfpl['seriesID'].str.split(pat = 'CT', expand = True)
test = test[1]
test = test.str[:7]
dfpl['GEO_ID'] = '1600000US' + test

In [11]:
df = dfco.append(dfpl)

In [12]:
geos.head()

,NAME,GEO_ID
0,"Cheatham County, Tennessee",0500000US47021
1,"Davidson County, Tennessee",0500000US47037
2,"Dickson County, Tennessee",0500000US47043
3,"Houston County, Tennessee",0500000US47083
4,"Humphreys County, Tennessee",0500000US47085


In [16]:
df = df.merge(geos, how = 'inner')

In [20]:
df = df.drop(columns = ['latest', 'footnotes', 'seriesID'])

In [21]:
df.tail()

,year,period,periodName,value,GEO_ID,NAME
1329,2019,M06,June,2.8,1600000US4770580,"Spring Hill city, Tennessee"
1330,2019,M05,May,2.3,1600000US4770580,"Spring Hill city, Tennessee"
1331,2019,M04,April,2.5,1600000US4770580,"Spring Hill city, Tennessee"
1332,2019,M03,March,2.6,1600000US4770580,"Spring Hill city, Tennessee"
1333,2019,M02,February,2.5,1600000US4770580,"Spring Hill city, Tennessee"


In [25]:
df_annual = df.loc[(df['periodName'] == 'Annual')&(df['year'] == '2021')].reset_index(drop = True)
df_annual = df_annual.drop(columns = ['period', 'periodName']).rename(columns = {'year':'Year', 'value':'LAUS:Unemployment Rate'})

In [26]:
df_annual.head()

,Year,LAUS:Unemployment Rate,GEO_ID,NAME
0,2021,3.1,0500000US47021,"Cheatham County, Tennessee"
1,2021,4.2,0500000US47037,"Davidson County, Tennessee"
2,2021,3.4,0500000US47043,"Dickson County, Tennessee"
3,2021,4.8,0500000US47083,"Houston County, Tennessee"
4,2021,3.9,0500000US47085,"Humphreys County, Tennessee"


In [27]:
df_monthly = df.loc[(df['periodName'] != 'Annual')].reset_index(drop = True)
df_monthly = df_monthly.rename(columns = {'period':'Month', 'periodName': 'Month Name','year':'Year', 'value':'LAUS:Unemployment Rate'})

In [28]:
df_monthly['Month'] = df_monthly['Month'].str.slice(start = 1).astype(float)

In [29]:
df_monthly.head(10)

,Year,Month,Month Name,LAUS:Unemployment Rate,GEO_ID,NAME
0,2022,8.0,August,2.5,0500000US47021,"Cheatham County, Tennessee"
1,2022,7.0,July,2.8,0500000US47021,"Cheatham County, Tennessee"
2,2022,6.0,June,3.2,0500000US47021,"Cheatham County, Tennessee"
3,2022,5.0,May,2.6,0500000US47021,"Cheatham County, Tennessee"
4,2022,4.0,April,2.5,0500000US47021,"Cheatham County, Tennessee"
5,2022,3.0,March,2.4,0500000US47021,"Cheatham County, Tennessee"
6,2022,2.0,February,2.5,0500000US47021,"Cheatham County, Tennessee"
7,2022,1.0,January,2.6,0500000US47021,"Cheatham County, Tennessee"
8,2021,12.0,December,2.3,0500000US47021,"Cheatham County, Tennessee"
9,2021,11.0,November,2.2,0500000US47021,"Cheatham County, Tennessee"


In [30]:
df_annual.to_csv('../../Outputs/LAUSANNUAL.csv', index = False)

In [31]:
df_monthly.to_csv('../../Outputs/LAUSMONTHLY.csv', index = False)